In [1]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
IMAGES_DIR = 'cv_open_dataset/open_img'  # Путь к вашему датасету с изображениями
MASKS_DIR = 'cv_open_dataset/open_msk_new'  # Путь к вашему датасету с масками
OUTPUT_DIR = './datasets/train_data'  # Путь к выходной директории
TRAIN_SIZE = 0.8  # Процент обучающей выборки

In [3]:
IMAGES_DIR = 'cv_open_dataset/open_img_synt'  # Путь к вашему датасету с изображениями
MASKS_DIR = 'cv_open_dataset/open_msk_synt_new'  # Путь к вашему датасету с масками
OUTPUT_DIR = './datasets/train_data_synt'  # Путь к выходной директории
TRAIN_SIZE = 0.8  # Процент обучающей выборки

In [4]:
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

In [5]:
image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(MASKS_DIR) if f.endswith('.png')]
if len(image_files) != len(mask_files):
    print(len(image_files))
    print(len(mask_files))
    print("Количество изображений и масок не совпадает.")
    for mask in mask_files:
        if mask.replace(".png", ".jpg") not in image_files:
            print(mask)

In [6]:
train_images, val_images = train_test_split(image_files, train_size=TRAIN_SIZE, random_state=42)

In [7]:
def convert_mask_to_yolo(mask_path, out):
    # Открываем изображение
    image = cv2.imread(mask_path)

    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return
    
    height, width = image.shape[:2]

    # Создаем маску для черного цвета
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))

    # Создаем новое изображение, где черный цвет остается, а остальные цвета становятся белыми
    new_image = np.ones_like(image) * 255  # Начинаем с белого изображения
    new_image[black_mask > 0] = [0, 0, 0]  # Заменяем черные пиксели

    # Преобразуем в градации серого для нахождения контуров
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Находим контуры
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Сохраняем контуры в текстовом формате
    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, out, output_file_name)

    with open(output_file_path, 'w') as f:
        for index, contour in enumerate(contours):
            # Получаем координаты всех точек контура
            contour_points = contour.reshape(-1, 2)
            # Нормализуем координаты
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

In [8]:
# Копирование изображений и масок в соответствующие папки
for img in train_images:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/train', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/train')

In [9]:
for img in val_images:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/val', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/val')


In [10]:
data_yaml_content = f"""
train: train_data_synt/images/train
val: train_data_synt/images/val

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("Датасет успешно разбит и сохранен в структуре проекта.")

Датасет успешно разбит и сохранен в структуре проекта.


In [11]:
model = YOLO("yolo11n-seg.pt")

In [14]:
train_results = model.train(
    data="./data.yaml",  # path to dataset YAML
    epochs=20,  # number of training epochs
    imgsz=640,  # training image size
    device="0",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    
)

Ultralytics 8.3.44  Python-3.10.0 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=./data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train133, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning C:\Users\nadyl\Documents\GitHub\nornnickel_hac\datasets\train_data_synt\labels\train.cache... 253 images, 91 backgrounds, 0 corrupt: 100%|██████████| 253/253 [00:00<?, ?it/s]
val: Scanning C:\Users\nadyl\Documents\GitHub\nornnickel_hac\datasets\train_data_synt\labels\val.cache... 64 images, 20 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]


Plotting labels to c:\Users\nadyl\Documents\GitHub\nornnickel_hac\runs\segment\train133\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to c:\Users\nadyl\Documents\GitHub\nornnickel_hac\runs\segment\train133
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/20      3.71G      2.947      3.504      2.462      2.106        104        640: 100%|██████████| 16/16 [00:06<00:00,  2.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.90it/s]

                   all         64        674      0.264      0.194      0.109     0.0441      0.213       0.16     0.0772     0.0255


...

      20/20      3.21G      0.846      1.328     0.6002     0.9485         81        640: 100%|██████████| 16/16 [00:04<00:00,  3.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.11it/s]

                   all         64        674        0.9       0.81      0.838      0.612      0.903      0.813      0.837      0.543



20 epochs completed in 0.041 hours.
Optimizer stripped from c:\Users\nadyl\Documents\GitHub\nornnickel_hac\runs\segment\train133\weights\last.pt, 6.0MB
Optimizer stripped from c:\Users\nadyl\Documents\GitHub\nornnickel_hac\runs\segment\train133\weights\best.pt, 6.0MB

Validating c:\Users\nadyl\Documents\GitHub\nornnickel_hac\runs\segment\train133\weights\best.pt...
Ultralytics 8.3.44  Python-3.10.0 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.70it/s]


                   all         64        674      0.898      0.812      0.839      0.612      0.902      0.815      0.837      0.542
Speed: 0.8ms preprocess, 2.3ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to c:\Users\nadyl\Documents\GitHub\nornnickel_hac\runs\segment\train133


In [ ]:
results = model.val(data="./data.yaml")
# print(results)

Ultralytics 8.3.44  Python-3.10.0 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


val: Scanning C:\Users\nadyl\Documents\GitHub\nornnickel_hac\datasets\train_data_synt\labels\val.cache... 64 images, 20 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:06<00:00,  1.72s/it]


                   all         64        674      0.895      0.845      0.863      0.674      0.888      0.832      0.848      0.565
Speed: 6.9ms preprocess, 21.8ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to c:\Users\nadyl\Documents\GitHub\nornnickel_hac\runs\segment\train132
ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001F783E13C70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018, 

In [7]:
def infer_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)

    # Инференс
    return model(image)



In [ ]:
# Функция для создания маски с черным фоном
def create_mask(image_path, results):
    # Загружаем изображение и переводим в градации серого
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Создаем пустую маску с черным фоном
    mask = np.zeros((height, width), dtype=np.uint8)

    # Проходим по результатам и создаем маску
    for result in results:
        masks = result.masks  # Получаем маски из результатов
        if masks is not None:
            for mask_array in masks.data:  # Получаем маски как массивы
                mask_i = mask_array.numpy()  # Преобразуем маску в numpy массив
                
                # Изменяем размер маски под размер оригинального изображения
                mask_i_resized = cv2.resize(mask_i, (width, height), interpolation=cv2.INTER_LINEAR)
                
                # Накладываем маску на пустую маску (255 для белого)
                mask[mask_i_resized > 0] = 255

    return mask

In [ ]:
results = infer_image("./cv_test_dataset/test_img/_08.jpg")
mask_image = create_mask("./cv_test_dataset/test_img/_08.jpg", results)

# Сохраняем маску в формате PNG
mask_output_path = './mask_image.png'  # Укажите путь для сохранения маски
cv2.imwrite(mask_output_path, mask_image)

0: 384x640 3 contaminateds, 71.0ms
Speed: 3.0ms preprocess, 71.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


True